### Import Packages & Define Constants:

In [5]:
from google.colab import drive
drive.mount('/content/drive')
# drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!ls

drive  sample_data


In [1]:
import torch
from torch.autograd import Variable
from torch import optim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import time

In [2]:
R_max = 5 # Maximum rating value possible
M = 49289  # No. of users in Epinions Dataset
N = 139738  # No. of items in Epinions Dataset
D = 10 # Dimensionality of latent space for users and items
lamda = 0.001 # Hyperparameter for loss function

### Read Epinions Dataset into Ratings Matrix and Trust Matrix:

In [ ]:
ratings_file = "drive/My Drive/SCFT_Project/data/epinions_ratings_data.txt"
trust_file = "drive/My Drive/SCFT_Project/data/standardized_epinions_trust_data.txt"

In [ ]:
def read_ratings_data():
    global ratings_file
    ratings_data = pd.read_csv(ratings_file, sep=" ", header=None)
    ratings_data.columns = ["user_id", "item_id", "rating"]
    return ratings_data

In [ ]:
ratings_data = read_ratings_data()

In [ ]:
ratings_data["rating"] /= R_max
convert_dict = {"user_id": int,
                "item_id": int,
                "rating": float}
ratings_data = ratings_data.astype(convert_dict)
print(ratings_data.dtypes)

user_id      int64
item_id      int64
rating     float64
dtype: object


In [ ]:
ratings_data.head(4)

,user_id,item_id,rating
0,1,101,1.0
1,1,102,0.6
2,1,10,0.6
3,1,103,1.0


In [ ]:
ratings_data.tail(4)

,user_id,item_id,rating
664819,49289,3862,0.8
664820,49289,3939,1.0
664821,49289,60213,0.8
664822,49289,62722,0.8


In [ ]:
def read_trust_data():
    global trust_file
    trust_data = pd.read_csv(trust_file, sep=" ", header=None)
    trust_data.columns = ["user1_id", "user2_id", "trust_val"]
    return trust_data

In [ ]:
trust_data = read_trust_data()

In [ ]:
trust_data.head(4)

,user1_id,user2_id,trust_val
0,22605,42915,1
1,22605,5052,1
2,22605,42913,1
3,22605,18420,1


In [ ]:
trust_data.tail(4)

,user1_id,user2_id,trust_val
487179,36960,1056,1
487180,36960,422,1
487181,36960,804,1
487182,36960,12784,1


In [ ]:
TRAIN_RATIO = 0.80 # Ratio of training to validation set

In [ ]:
ratings_data_train, ratings_data_test = train_test_split(ratings_data, train_size=TRAIN_RATIO, shuffle=True, random_state=19)

In [ ]:
print(len(ratings_data_train))
print(len(ratings_data_test))

531858
132965


In [ ]:
ratings_data_train.head(5)

,user_id,item_id,rating
94467,804,56181,0.8
828,11,824,1.0
382104,9866,43033,1.0
584599,29534,94653,1.0
449167,14368,9393,1.0


In [ ]:
ratings_data_train.tail(5)

,user_id,item_id,rating
480168,16803,126965,1.0
623970,37756,2422,0.8
82677,715,2956,0.8
207470,2730,49213,0.8
521821,21081,130213,0.8


In [ ]:
ratings_data_test.head(5)

,user_id,item_id,rating
66340,501,6864,1.0
46401,375,1072,0.6
102744,953,59912,1.0
207332,2727,52999,1.0
429791,12926,2750,0.8


In [ ]:
ratings_data_test.tail(5)

,user_id,item_id,rating
360986,8740,88696,0.8
500537,18774,8533,1.0
592152,30754,61116,1.0
385853,10057,96342,0.8
622059,37355,58284,0.8


### Train Truster Model:

In [ ]:
torch.manual_seed(42) # Set any random seed for reproducibility
# Browse Reviews Behaviour Matrix (Influence from other users)- Approximates User feature matrix U in Truster Model
B_trusterMF = Variable(torch.empty((D, M)).normal_(mean=0.0, std=0.1), requires_grad=True) 
# Item feature matrix
V_trusterMF = Variable(torch.empty((D, N)).normal_(mean=0.0, std=0.1), requires_grad=True) 
# Write Reviews Behaviour Matrix (Influence other users)
W_trusterMF = Variable(torch.empty((D, M)).normal_(mean=0.0, std=0.1), requires_grad=True)

In [ ]:
# nbi - no. of ratings given by user i
# nvj - no. of ratings given to item j
def calculate_nbi_and_nvj():
    global ratings_data_train, M, N
    nbi = np.zeros(M)
    nvj = np.zeros(N)
    for idx in range(len(ratings_data_train)):
        nbi[int(ratings_data_train.iloc[idx]["user_id"])-1] += 1
        nvj[int(ratings_data_train.iloc[idx]["item_id"])-1] += 1
    return nbi, nvj

In [ ]:
# mbi - no. of users who are trusted by user i
# mwk - no. of users who trust user k
def calculate_mbi_and_mwk():
    global trust_data, M
    mbi = np.zeros(M)
    mwk = np.zeros(M)
    for idx in range(len(trust_data)):
        mbi[trust_data.iloc[idx]["user1_id"]-1] += 1
        mwk[trust_data.iloc[idx]["user2_id"]-1] += 1
    return mbi, mwk

In [ ]:
# start = time.time()
# nbi, nvj = calculate_nbi_and_nvj()
# end = time.time()
# print(end-start, " seconds")

In [ ]:
#np.save('drive/My Drive/SCFT_Project/npy_files/nbi.npy', nbi)
# # nbi = np.load('drive/My Drive/SCFT_Project/npy_files/nbi.npy')
#np.save('drive/My Drive/SCFT_Project/npy_files/nvj.npy', nvj)
# # nvj = np.load('drive/My Drive/SCFT_Project/npy_files/nvj.npy')

In [ ]:
# start = time.time()
# mbi, mwk = calculate_mbi_and_mwk()
# end = time.time()
# print(end-start, " seconds")

107.72999572753906  seconds


In [ ]:
# np.save('drive/My Drive/SCFT_Project/npy_files/mbi.npy', mbi)
# # mbi = np.load('drive/My Drive/SCFT_Project/npy_files/mbi.npy')
# np.save('drive/My Drive/SCFT_Project/npy_files/mwk.npy', mwk)
# # mwk = np.load('drive/My Drive/SCFT_Project/npy_files/mwk.npy')

In [ ]:
# nbi - no. of ratings given by user i
nbi = np.load('drive/My Drive/SCFT_Project/npy_files/nbi.npy')
# nvj - no. of ratings given to item j
nvj = np.load('drive/My Drive/SCFT_Project/npy_files/nvj.npy')
# mbi - no. of users who are trusted by user i
mbi = np.load('drive/My Drive/SCFT_Project/npy_files/mbi.npy')
# mwk - no. of users who trust user k
mwk = np.load('drive/My Drive/SCFT_Project/npy_files/mwk.npy')

In [ ]:
epochs = 2 # No. of epochs
alpha_lr = 0.005 # Learning rate
optimizer = optim.SGD([B_trusterMF, V_trusterMF, W_trusterMF], lr=alpha_lr)

In [ ]:
for iteration in range(0, 0+epochs):
    start = time.time()
    loss = torch.tensor(0.0)
    optimizer.zero_grad()
    for idx in range(len(ratings_data_train)):
        user_id = int(ratings_data_train.iloc[idx]["user_id"])
        item_id = int(ratings_data_train.iloc[idx]["item_id"])
        rating = ratings_data_train.iloc[idx]["rating"]
        loss += (torch.sigmoid(torch.dot(B_trusterMF[:, user_id-1], V_trusterMF[:, item_id-1])) - rating).pow(2)
    for idx in range(len(trust_data)):
        user1_id = trust_data.iloc[idx]["user1_id"]
        user2_id = trust_data.iloc[idx]["user2_id"]
        trust_val = trust_data.iloc[idx]["trust_val"]
        loss += (torch.sigmoid(torch.dot(B_trusterMF[:, user1_id-1], W_trusterMF[:, user2_id-1])) - trust_val).pow(2)
    reg_loss = torch.tensor(0.0)
    for user_id in range(M):
        reg_loss += ((nbi[user_id]+mbi[user_id]) * (B_trusterMF[:, user_id].pow(2).sum()))
        reg_loss += (mwk[user_id] * (W_trusterMF[:, user_id].pow(2).sum()))
    for item_id in range(N):
        reg_loss += (nvj[item_id] * (V_trusterMF[:, item_id].pow(2).sum()))
    loss += (lamda * reg_loss)
    loss.backward() # Loss corresponding to Eq. (4) in the paper
    optimizer.step()
    end = time.time()
    print("Iteration: ", iteration, ", loss: ", loss, ", time(seconds): ", end-start)
    torch.save(B_trusterMF, "drive/My Drive/SCFT_Project/B_trusterMF_"+str(iteration)+".pth")
    torch.save(V_trusterMF, "drive/My Drive/SCFT_Project/V_trusterMF_"+str(iteration)+".pth")
    torch.save(W_trusterMF, "drive/My Drive/SCFT_Project/W_trusterMF_"+str(iteration)+".pth")

Iteration:  0 , loss:  tensor(200275.1406, grad_fn=<AddBackward0>) , time(seconds):  2167.779409646988
Iteration:  1 , loss:  tensor(200225.3594, grad_fn=<AddBackward0>) , time(seconds):  2113.917291879654


In [ ]:
B_trusterMF = Variable(torch.load("drive/My Drive/SCFT_Project/B_trusterMF_"+str(1)+".pth"), requires_grad=True)
V_trusterMF = Variable(torch.load("drive/My Drive/SCFT_Project/V_trusterMF_"+str(1)+".pth"), requires_grad=True)
W_trusterMF = Variable(torch.load("drive/My Drive/SCFT_Project/W_trusterMF_"+str(1)+".pth"), requires_grad=True)

In [ ]:
nbi = np.load('drive/My Drive/SCFT_Project/npy_files/nbi.npy')
nvj = np.load('drive/My Drive/SCFT_Project/npy_files/nvj.npy')
mbi = np.load('drive/My Drive/SCFT_Project/npy_files/mbi.npy')
mwk = np.load('drive/My Drive/SCFT_Project/npy_files/mwk.npy')

In [ ]:
epochs = 4
alpha_lr = 0.4
optimizer = optim.SGD([B_trusterMF, V_trusterMF, W_trusterMF], lr=alpha_lr)

In [ ]:
for iteration in range(2, 2+epochs):
    start = time.time()
    loss = torch.tensor(0.0)
    optimizer.zero_grad()
    for idx in range(len(ratings_data_train)):
        user_id = int(ratings_data_train.iloc[idx]["user_id"])
        item_id = int(ratings_data_train.iloc[idx]["item_id"])
        rating = ratings_data_train.iloc[idx]["rating"]
        loss += (torch.sigmoid(torch.dot(B_trusterMF[:, user_id-1], V_trusterMF[:, item_id-1])) - rating).pow(2)
    for idx in range(len(trust_data)):
        user1_id = trust_data.iloc[idx]["user1_id"]
        user2_id = trust_data.iloc[idx]["user2_id"]
        trust_val = trust_data.iloc[idx]["trust_val"]
        loss += (torch.sigmoid(torch.dot(B_trusterMF[:, user1_id-1], W_trusterMF[:, user2_id-1])) - trust_val).pow(2)
    reg_loss = torch.tensor(0.0)
    for user_id in range(M):
        reg_loss += ((nbi[user_id]+mbi[user_id]) * (B_trusterMF[:, user_id].pow(2).sum()))
        reg_loss += (mwk[user_id] * (W_trusterMF[:, user_id].pow(2).sum()))
    for item_id in range(N):
        reg_loss += (nvj[item_id] * (V_trusterMF[:, item_id].pow(2).sum()))
    loss += (lamda * reg_loss)
    loss.backward() # Loss corresponding to Eq. (4) in the paper
    optimizer.step()
    end = time.time()
    print("Iteration: ", iteration, ", loss: ", loss, ", time(seconds): ", end-start)
    torch.save(B_trusterMF, "drive/My Drive/SCFT_Project/B_trusterMF_"+str(iteration)+".pth")
    torch.save(V_trusterMF, "drive/My Drive/SCFT_Project/V_trusterMF_"+str(iteration)+".pth")
    torch.save(W_trusterMF, "drive/My Drive/SCFT_Project/W_trusterMF_"+str(iteration)+".pth")

Iteration:  2 , loss:  tensor(200174.4219, grad_fn=<AddBackward0>) , time(seconds):  2108.184391260147
Iteration:  3 , loss:  tensor(195684.8125, grad_fn=<AddBackward0>) , time(seconds):  2214.1749184131622
Iteration:  4 , loss:  tensor(146265.5469, grad_fn=<AddBackward0>) , time(seconds):  2165.8209903240204
Iteration:  5 , loss:  tensor(93137.5469, grad_fn=<AddBackward0>) , time(seconds):  2170.444637298584


In [ ]:
B_trusterMF = Variable(torch.load("drive/My Drive/SCFT_Project/B_trusterMF_"+str(5)+".pth"), requires_grad=True)
V_trusterMF = Variable(torch.load("drive/My Drive/SCFT_Project/V_trusterMF_"+str(5)+".pth"), requires_grad=True)
W_trusterMF = Variable(torch.load("drive/My Drive/SCFT_Project/W_trusterMF_"+str(5)+".pth"), requires_grad=True)
nbi = np.load('drive/My Drive/SCFT_Project/npy_files/nbi.npy')
nvj = np.load('drive/My Drive/SCFT_Project/npy_files/nvj.npy')
mbi = np.load('drive/My Drive/SCFT_Project/npy_files/mbi.npy')
mwk = np.load('drive/My Drive/SCFT_Project/npy_files/mwk.npy')

In [ ]:
epochs = 2
alpha_lr = 0.4
optimizer = optim.SGD([B_trusterMF, V_trusterMF, W_trusterMF], lr=alpha_lr)

In [ ]:
for iteration in range(6, 6+epochs):
    start = time.time()
    loss = torch.tensor(0.0)
    optimizer.zero_grad()
    for idx in range(len(ratings_data_train)):
        user_id = int(ratings_data_train.iloc[idx]["user_id"])
        item_id = int(ratings_data_train.iloc[idx]["item_id"])
        rating = ratings_data_train.iloc[idx]["rating"]
        loss += (torch.sigmoid(torch.dot(B_trusterMF[:, user_id-1], V_trusterMF[:, item_id-1])) - rating).pow(2)
    for idx in range(len(trust_data)):
        user1_id = trust_data.iloc[idx]["user1_id"]
        user2_id = trust_data.iloc[idx]["user2_id"]
        trust_val = trust_data.iloc[idx]["trust_val"]
        loss += (torch.sigmoid(torch.dot(B_trusterMF[:, user1_id-1], W_trusterMF[:, user2_id-1])) - trust_val).pow(2)
    reg_loss = torch.tensor(0.0)
    for user_id in range(M):
        reg_loss += ((nbi[user_id]+mbi[user_id]) * (B_trusterMF[:, user_id].pow(2).sum()))
        reg_loss += (mwk[user_id] * (W_trusterMF[:, user_id].pow(2).sum()))
    for item_id in range(N):
        reg_loss += (nvj[item_id] * (V_trusterMF[:, item_id].pow(2).sum()))
    loss += (lamda * reg_loss)
    loss.backward() # Loss corresponding to Eq. (4) in the paper
    optimizer.step()
    end = time.time()
    print("Iteration: ", iteration, ", loss: ", loss, ", time(seconds): ", end-start)
    torch.save(B_trusterMF, "drive/My Drive/SCFT_Project/B_trusterMF_"+str(iteration)+".pth")
    torch.save(V_trusterMF, "drive/My Drive/SCFT_Project/V_trusterMF_"+str(iteration)+".pth")
    torch.save(W_trusterMF, "drive/My Drive/SCFT_Project/W_trusterMF_"+str(iteration)+".pth")

In [ ]:
epochs = 2
alpha_lr = 0.25
optimizer = optim.SGD([B_trusterMF, V_trusterMF, W_trusterMF], lr=alpha_lr)

In [ ]:
for iteration in range(8, 8+epochs):
    start = time.time()
    loss = torch.tensor(0.0)
    optimizer.zero_grad()
    for idx in range(len(ratings_data_train)):
        user_id = int(ratings_data_train.iloc[idx]["user_id"])
        item_id = int(ratings_data_train.iloc[idx]["item_id"])
        rating = ratings_data_train.iloc[idx]["rating"]
        loss += (torch.sigmoid(torch.dot(B_trusterMF[:, user_id-1], V_trusterMF[:, item_id-1])) - rating).pow(2)
    for idx in range(len(trust_data)):
        user1_id = trust_data.iloc[idx]["user1_id"]
        user2_id = trust_data.iloc[idx]["user2_id"]
        trust_val = trust_data.iloc[idx]["trust_val"]
        loss += (torch.sigmoid(torch.dot(B_trusterMF[:, user1_id-1], W_trusterMF[:, user2_id-1])) - trust_val).pow(2)
    reg_loss = torch.tensor(0.0)
    for user_id in range(M):
        reg_loss += ((nbi[user_id]+mbi[user_id]) * (B_trusterMF[:, user_id].pow(2).sum()))
        reg_loss += (mwk[user_id] * (W_trusterMF[:, user_id].pow(2).sum()))
    for item_id in range(N):
        reg_loss += (nvj[item_id] * (V_trusterMF[:, item_id].pow(2).sum()))
    loss += (lamda * reg_loss)
    loss.backward() # Loss corresponding to Eq. (4) in the paper
    optimizer.step()
    end = time.time()
    print("Iteration: ", iteration, ", loss: ", loss, ", time(seconds): ", end-start)
    torch.save(B_trusterMF, "drive/My Drive/SCFT_Project/B_trusterMF_"+str(iteration)+".pth")
    torch.save(V_trusterMF, "drive/My Drive/SCFT_Project/V_trusterMF_"+str(iteration)+".pth")
    torch.save(W_trusterMF, "drive/My Drive/SCFT_Project/W_trusterMF_"+str(iteration)+".pth")

Iteration:  8 , loss:  tensor(150490.4844, grad_fn=<AddBackward0>) , time(seconds):  4272.635210514069
Iteration:  9 , loss:  tensor(77514.4375, grad_fn=<AddBackward0>) , time(seconds):  4330.430537939072


In [ ]:
epochs = 2

In [ ]:
for iteration in range(10, 10+epochs):
    start = time.time()
    loss = torch.tensor(0.0)
    optimizer.zero_grad()
    for idx in range(len(ratings_data_train)):
        user_id = int(ratings_data_train.iloc[idx]["user_id"])
        item_id = int(ratings_data_train.iloc[idx]["item_id"])
        rating = ratings_data_train.iloc[idx]["rating"]
        loss += (torch.sigmoid(torch.dot(B_trusterMF[:, user_id-1], V_trusterMF[:, item_id-1])) - rating).pow(2)
    for idx in range(len(trust_data)):
        user1_id = trust_data.iloc[idx]["user1_id"]
        user2_id = trust_data.iloc[idx]["user2_id"]
        trust_val = trust_data.iloc[idx]["trust_val"]
        loss += (torch.sigmoid(torch.dot(B_trusterMF[:, user1_id-1], W_trusterMF[:, user2_id-1])) - trust_val).pow(2)
    reg_loss = torch.tensor(0.0)
    for user_id in range(M):
        reg_loss += ((nbi[user_id]+mbi[user_id]) * (B_trusterMF[:, user_id].pow(2).sum()))
        reg_loss += (mwk[user_id] * (W_trusterMF[:, user_id].pow(2).sum()))
    for item_id in range(N):
        reg_loss += (nvj[item_id] * (V_trusterMF[:, item_id].pow(2).sum()))
    loss += (lamda * reg_loss)
    loss.backward() # Loss corresponding to Eq. (4) in the paper
    optimizer.step()
    end = time.time()
    print("Iteration: ", iteration, ", loss: ", loss, ", time(seconds): ", end-start)
    torch.save(B_trusterMF, "drive/My Drive/SCFT_Project/B_trusterMF_"+str(iteration)+".pth")
    torch.save(V_trusterMF, "drive/My Drive/SCFT_Project/V_trusterMF_"+str(iteration)+".pth")
    torch.save(W_trusterMF, "drive/My Drive/SCFT_Project/W_trusterMF_"+str(iteration)+".pth")

Iteration:  10 , loss:  tensor(56805.3047, grad_fn=<AddBackward0>) , time(seconds):  4252.879674196243
Iteration:  11 , loss:  tensor(47447.3828, grad_fn=<AddBackward0>) , time(seconds):  4283.094748258591


In [ ]:
torch.save(optimizer.state_dict(), "drive/My Drive/SCFT_Project/optimizer_trusterMF_11.pth")

In [ ]:
epochs = 2

In [ ]:
for iteration in range(12, 12+epochs):
    start = time.time()
    loss = torch.tensor(0.0)
    optimizer.zero_grad()
    for idx in range(len(ratings_data_train)):
        user_id = int(ratings_data_train.iloc[idx]["user_id"])
        item_id = int(ratings_data_train.iloc[idx]["item_id"])
        rating = ratings_data_train.iloc[idx]["rating"]
        loss += (torch.sigmoid(torch.dot(B_trusterMF[:, user_id-1], V_trusterMF[:, item_id-1])) - rating).pow(2)
    for idx in range(len(trust_data)):
        user1_id = trust_data.iloc[idx]["user1_id"]
        user2_id = trust_data.iloc[idx]["user2_id"]
        trust_val = trust_data.iloc[idx]["trust_val"]
        loss += (torch.sigmoid(torch.dot(B_trusterMF[:, user1_id-1], W_trusterMF[:, user2_id-1])) - trust_val).pow(2)
    reg_loss = torch.tensor(0.0)
    for user_id in range(M):
        reg_loss += ((nbi[user_id]+mbi[user_id]) * (B_trusterMF[:, user_id].pow(2).sum()))
        reg_loss += (mwk[user_id] * (W_trusterMF[:, user_id].pow(2).sum()))
    for item_id in range(N):
        reg_loss += (nvj[item_id] * (V_trusterMF[:, item_id].pow(2).sum()))
    loss += (lamda * reg_loss)
    loss.backward() # Loss corresponding to Eq. (4) in the paper
    optimizer.step()
    end = time.time()
    print("Iteration: ", iteration, ", loss: ", loss, ", time(seconds): ", end-start)
    torch.save(B_trusterMF, "drive/My Drive/SCFT_Project/B_trusterMF_"+str(iteration)+".pth")
    torch.save(V_trusterMF, "drive/My Drive/SCFT_Project/V_trusterMF_"+str(iteration)+".pth")
    torch.save(W_trusterMF, "drive/My Drive/SCFT_Project/W_trusterMF_"+str(iteration)+".pth")

Iteration:  12 , loss:  tensor(78582.5234, grad_fn=<AddBackward0>) , time(seconds):  4240.109281539917
Iteration:  13 , loss:  tensor(42432.4609, grad_fn=<AddBackward0>) , time(seconds):  4237.006747484207


In [ ]:
torch.save(optimizer.state_dict(), "drive/My Drive/SCFT_Project/optimizer_trusterMF_13.pth")

In [ ]:
nbi = np.load('drive/My Drive/SCFT_Project/npy_files/nbi.npy')
nvj = np.load('drive/My Drive/SCFT_Project/npy_files/nvj.npy')
mbi = np.load('drive/My Drive/SCFT_Project/npy_files/mbi.npy')
mwk = np.load('drive/My Drive/SCFT_Project/npy_files/mwk.npy')
B_trusterMF = Variable(torch.load("drive/My Drive/SCFT_Project/B_trusterMF_"+str(13)+".pth"), requires_grad=True)
V_trusterMF = Variable(torch.load("drive/My Drive/SCFT_Project/V_trusterMF_"+str(13)+".pth"), requires_grad=True)
W_trusterMF = Variable(torch.load("drive/My Drive/SCFT_Project/W_trusterMF_"+str(13)+".pth"), requires_grad=True)

In [ ]:
epochs = 3
alpha_lr = 0.20
optimizer = optim.SGD([B_trusterMF, V_trusterMF, W_trusterMF], lr=alpha_lr)
checkpt = torch.load("drive/My Drive/SCFT_Project/optimizer_trusterMF_13.pth")
optimizer.load_state_dict(checkpt)
print(optimizer.state_dict())
for pg in optimizer.param_groups:
    pg['lr'] = alpha_lr
print(optimizer.state_dict())

{'state': {}, 'param_groups': [{'lr': 0.25, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [139718035755296, 139718035711896, 139718035712112]}]}
{'state': {}, 'param_groups': [{'lr': 0.2, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [139718035755296, 139718035711896, 139718035712112]}]}


In [ ]:
for iteration in range(14, 17):
    start = time.time()
    loss = torch.tensor(0.0)
    optimizer.zero_grad()
    for idx in range(len(ratings_data_train)):
        user_id = int(ratings_data_train.iloc[idx]["user_id"])
        item_id = int(ratings_data_train.iloc[idx]["item_id"])
        rating = ratings_data_train.iloc[idx]["rating"]
        loss += (torch.sigmoid(torch.dot(B_trusterMF[:, user_id-1], V_trusterMF[:, item_id-1])) - rating).pow(2)
    for idx in range(len(trust_data)):
        user1_id = trust_data.iloc[idx]["user1_id"]
        user2_id = trust_data.iloc[idx]["user2_id"]
        trust_val = trust_data.iloc[idx]["trust_val"]
        loss += (torch.sigmoid(torch.dot(B_trusterMF[:, user1_id-1], W_trusterMF[:, user2_id-1])) - trust_val).pow(2)
    reg_loss = torch.tensor(0.0)
    for user_id in range(M):
        reg_loss += ((nbi[user_id]+mbi[user_id]) * (B_trusterMF[:, user_id].pow(2).sum()))
        reg_loss += (mwk[user_id] * (W_trusterMF[:, user_id].pow(2).sum()))
    for item_id in range(N):
        reg_loss += (nvj[item_id] * (V_trusterMF[:, item_id].pow(2).sum()))
    loss += (lamda * reg_loss)
    loss.backward() # Loss corresponding to Eq. (4) in the paper
    optimizer.step()
    end = time.time()
    print("Iteration: ", iteration, ", loss: ", loss, ", time(seconds): ", end-start)
    torch.save(B_trusterMF, "drive/My Drive/SCFT_Project/B_trusterMF_"+str(iteration)+".pth")
    torch.save(V_trusterMF, "drive/My Drive/SCFT_Project/V_trusterMF_"+str(iteration)+".pth")
    torch.save(W_trusterMF, "drive/My Drive/SCFT_Project/W_trusterMF_"+str(iteration)+".pth")

Iteration:  14 , loss:  tensor(52122.6172, grad_fn=<AddBackward0>) , time(seconds):  4210.962449550629
Iteration:  15 , loss:  tensor(47897.1992, grad_fn=<AddBackward0>) , time(seconds):  4222.90231347084
Iteration:  16 , loss:  tensor(254963.1562, grad_fn=<AddBackward0>) , time(seconds):  3934.3916993141174


In [ ]:
torch.save(optimizer.state_dict(), "drive/My Drive/SCFT_Project/optimizer_trusterMF_16.pth")

### Train Trustee Model:

In [ ]:
torch.manual_seed(42) # Set any random seed for reproducibility
# Browse Reviews Behaviour Matrix (Influence from other users)
B_trusteeMF = Variable(torch.empty((D, M)).normal_(mean=0.0, std=0.1), requires_grad=True)
# Item feature matrix
V_trusteeMF = Variable(torch.empty((D, N)).normal_(mean=0.0, std=0.1), requires_grad=True)
# Write Reviews Behaviour Matrix (Influence other users) - Approximates User feature matrix U in Trustee Model
W_trusteeMF = Variable(torch.empty((D, M)).normal_(mean=0.0, std=0.1), requires_grad=True)

In [ ]:
# nwi - no. of ratings given by user i
# nvj - no. of ratings given to item j
def calculate_nwi_and_nvj():
    global ratings_data_train, M, N
    nwi = np.zeros(M)
    nvj = np.zeros(N)
    for idx in range(len(ratings_data_train)):
        nwi[int(ratings_data_train.iloc[idx]["user_id"])-1] += 1
        nvj[int(ratings_data_train.iloc[idx]["item_id"])-1] += 1
    return nwi, nvj

In [ ]:
# mbk - no. of users who are trusted by user k
# mwi - no. of users who trust user i
def calculate_mwi_and_mbk():
    global trust_data, M
    mwi = np.zeros(M)
    mbk = np.zeros(M)
    for idx in range(len(trust_data)):
        mwi[trust_data.iloc[idx]["user2_id"]-1] += 1
        mbk[trust_data.iloc[idx]["user1_id"]-1] += 1
    return mwi, mbk

In [ ]:
start = time.time()
nwi, nvj = calculate_nwi_and_nvj()
end = time.time()
print(end-start, " seconds")

145.4626820087433  seconds


In [ ]:
np.save('drive/My Drive/SCFT_Project/npy_files/nwi.npy', nwi)
# nwi = np.load('drive/My Drive/SCFT_Project/npy_files/nwi.npy')
np.save('drive/My Drive/SCFT_Project/npy_files/nvj.npy', nvj)
# nvj = np.load('drive/My Drive/SCFT_Project/npy_files/nvj.npy')

In [ ]:
start = time.time()
mwi, mbk = calculate_mwi_and_mbk()
end = time.time()
print(end-start, " seconds")

97.13806438446045  seconds


In [ ]:
np.save('drive/My Drive/SCFT_Project/npy_files/mwi.npy', mwi)
# mwi = np.load('drive/My Drive/SCFT_Project/npy_files/mwi.npy')
np.save('drive/My Drive/SCFT_Project/npy_files/mbk.npy', mbk)
# mbk = np.load('drive/My Drive/SCFT_Project/npy_files/mbk.npy')

In [ ]:
epochs = 6 # No. of epochs
alpha_lr = 0.4 # Learning Rate
optimizer = optim.SGD([B_trusteeMF, V_trusteeMF, W_trusteeMF], lr=alpha_lr)

In [ ]:
for iteration in range(0, 0+epochs):
    start = time.time()
    loss = torch.tensor(0.0)
    optimizer.zero_grad()
    for idx in range(len(ratings_data_train)):
        user_id = int(ratings_data_train.iloc[idx]["user_id"])
        item_id = int(ratings_data_train.iloc[idx]["item_id"])
        rating = ratings_data_train.iloc[idx]["rating"]
        loss += (torch.sigmoid(torch.dot(W_trusteeMF[:, user_id-1], V_trusteeMF[:, item_id-1])) - rating).pow(2)
    for idx in range(len(trust_data)):
        user1_id = trust_data.iloc[idx]["user1_id"]
        user2_id = trust_data.iloc[idx]["user2_id"]
        trust_val = trust_data.iloc[idx]["trust_val"]
        loss += (torch.sigmoid(torch.dot(B_trusteeMF[:, user1_id-1], W_trusteeMF[:, user2_id-1])) - trust_val).pow(2)
    reg_loss = torch.tensor(0.0)
    for user_id in range(M):
        reg_loss += ((nwi[user_id]+mwi[user_id]) * (W_trusteeMF[:, user_id].pow(2).sum()))
        reg_loss += (mbk[user_id] * (B_trusteeMF[:, user_id].pow(2).sum()))
    for item_id in range(N):
        reg_loss += (nvj[item_id] * (V_trusteeMF[:, item_id].pow(2).sum()))
    loss += (lamda * reg_loss)
    loss.backward() # Loss corresponding to Eq. (5) in the paper
    optimizer.step()
    end = time.time()
    print("Iteration: ", iteration, ", loss: ", loss, ", time(seconds): ", end-start)
    torch.save(B_trusteeMF, "drive/My Drive/SCFT_Project/B_trusteeMF_"+str(iteration)+".pth")
    torch.save(V_trusteeMF, "drive/My Drive/SCFT_Project/V_trusteeMF_"+str(iteration)+".pth")
    torch.save(W_trusteeMF, "drive/My Drive/SCFT_Project/W_trusteeMF_"+str(iteration)+".pth")

In [ ]:
torch.save(optimizer.state_dict(), "drive/My Drive/SCFT_Project/optimizer_trusteeMF_5.pth")

In [ ]:
epochs = 4
alpha_lr = 0.25
for pg in optimizer.param_groups:
    pg['lr'] = alpha_lr

In [ ]:
for iteration in range(6, 6+epochs):
    start = time.time()
    loss = torch.tensor(0.0)
    optimizer.zero_grad()
    for idx in range(len(ratings_data_train)):
        user_id = int(ratings_data_train.iloc[idx]["user_id"])
        item_id = int(ratings_data_train.iloc[idx]["item_id"])
        rating = ratings_data_train.iloc[idx]["rating"]
        loss += (torch.sigmoid(torch.dot(W_trusteeMF[:, user_id-1], V_trusteeMF[:, item_id-1])) - rating).pow(2)
    for idx in range(len(trust_data)):
        user1_id = trust_data.iloc[idx]["user1_id"]
        user2_id = trust_data.iloc[idx]["user2_id"]
        trust_val = trust_data.iloc[idx]["trust_val"]
        loss += (torch.sigmoid(torch.dot(B_trusteeMF[:, user1_id-1], W_trusteeMF[:, user2_id-1])) - trust_val).pow(2)
    reg_loss = torch.tensor(0.0)
    for user_id in range(M):
        reg_loss += ((nwi[user_id]+mwi[user_id]) * (W_trusteeMF[:, user_id].pow(2).sum()))
        reg_loss += (mbk[user_id] * (B_trusteeMF[:, user_id].pow(2).sum()))
    for item_id in range(N):
        reg_loss += (nvj[item_id] * (V_trusteeMF[:, item_id].pow(2).sum()))
    loss += (lamda * reg_loss)
    loss.backward() # Loss corresponding to Eq. (5) in the paper
    optimizer.step()
    end = time.time()
    print("Iteration: ", iteration, ", loss: ", loss, ", time(seconds): ", end-start)
    torch.save(B_trusteeMF, "drive/My Drive/SCFT_Project/B_trusteeMF_"+str(iteration)+".pth")
    torch.save(V_trusteeMF, "drive/My Drive/SCFT_Project/V_trusteeMF_"+str(iteration)+".pth")
    torch.save(W_trusteeMF, "drive/My Drive/SCFT_Project/W_trusteeMF_"+str(iteration)+".pth")

Iteration:  6 , loss:  tensor(268725.0938, grad_fn=<AddBackward0>) , time(seconds):  4073.85556101799
Iteration:  7 , loss:  tensor(164052.6562, grad_fn=<AddBackward0>) , time(seconds):  4068.013741016388
Iteration:  8 , loss:  tensor(117212.2188, grad_fn=<AddBackward0>) , time(seconds):  4069.180951356888


In [ ]:
B_trusteeMF = Variable(torch.load("drive/My Drive/SCFT_Project/B_trusteeMF_"+str(8)+".pth"), requires_grad=True)
V_trusteeMF = Variable(torch.load("drive/My Drive/SCFT_Project/V_trusteeMF_"+str(8)+".pth"), requires_grad=True)
W_trusteeMF = Variable(torch.load("drive/My Drive/SCFT_Project/W_trusteeMF_"+str(8)+".pth"), requires_grad=True)
nwi = np.load('drive/My Drive/SCFT_Project/npy_files/nwi.npy')
nvj = np.load('drive/My Drive/SCFT_Project/npy_files/nvj.npy')
mwi = np.load('drive/My Drive/SCFT_Project/npy_files/mwi.npy')
mbk = np.load('drive/My Drive/SCFT_Project/npy_files/mbk.npy')

In [ ]:
epochs = 3
alpha_lr = 0.20
optimizer = optim.SGD([B_trusteeMF, V_trusteeMF, W_trusteeMF], lr=alpha_lr)
checkpt = torch.load("drive/My Drive/SCFT_Project/optimizer_trusteeMF_5.pth")
optimizer.load_state_dict(checkpt)
print(optimizer.state_dict())
for pg in optimizer.param_groups:
    pg['lr'] = alpha_lr
print(optimizer.state_dict())

{'state': {}, 'param_groups': [{'lr': 0.4, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [140037430628216, 140037464580032, 140037430669816]}]}
{'state': {}, 'param_groups': [{'lr': 0.2, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [140037430628216, 140037464580032, 140037430669816]}]}


In [ ]:
for iteration in range(9, 9+epochs):
    start = time.time()
    loss = torch.tensor(0.0)
    optimizer.zero_grad()
    for idx in range(len(ratings_data_train)):
        user_id = int(ratings_data_train.iloc[idx]["user_id"])
        item_id = int(ratings_data_train.iloc[idx]["item_id"])
        rating = ratings_data_train.iloc[idx]["rating"]
        loss += (torch.sigmoid(torch.dot(W_trusteeMF[:, user_id-1], V_trusteeMF[:, item_id-1])) - rating).pow(2)
    for idx in range(len(trust_data)):
        user1_id = trust_data.iloc[idx]["user1_id"]
        user2_id = trust_data.iloc[idx]["user2_id"]
        trust_val = trust_data.iloc[idx]["trust_val"]
        loss += (torch.sigmoid(torch.dot(B_trusteeMF[:, user1_id-1], W_trusteeMF[:, user2_id-1])) - trust_val).pow(2)
    reg_loss = torch.tensor(0.0)
    for user_id in range(M):
        reg_loss += ((nwi[user_id]+mwi[user_id]) * (W_trusteeMF[:, user_id].pow(2).sum()))
        reg_loss += (mbk[user_id] * (B_trusteeMF[:, user_id].pow(2).sum()))
    for item_id in range(N):
        reg_loss += (nvj[item_id] * (V_trusteeMF[:, item_id].pow(2).sum()))
    loss += (lamda * reg_loss)
    loss.backward() # Loss corresponding to Eq. (5) in the paper
    optimizer.step()
    end = time.time()
    print("Iteration: ", iteration, ", loss: ", loss, ", time(seconds): ", end-start)
    torch.save(B_trusteeMF, "drive/My Drive/SCFT_Project/B_trusteeMF_"+str(iteration)+".pth")
    torch.save(V_trusteeMF, "drive/My Drive/SCFT_Project/V_trusteeMF_"+str(iteration)+".pth")
    torch.save(W_trusteeMF, "drive/My Drive/SCFT_Project/W_trusteeMF_"+str(iteration)+".pth")

Iteration:  9 , loss:  tensor(81233.1875, grad_fn=<AddBackward0>) , time(seconds):  3971.718724489212
Iteration:  10 , loss:  tensor(67049.2891, grad_fn=<AddBackward0>) , time(seconds):  4145.7169761657715
Iteration:  11 , loss:  tensor(158793.0312, grad_fn=<AddBackward0>) , time(seconds):  4074.2037785053253


In [ ]:
B_trusteeMF = Variable(torch.load("drive/My Drive/SCFT_Project/B_trusteeMF_"+str(10)+".pth"), requires_grad=True)
V_trusteeMF = Variable(torch.load("drive/My Drive/SCFT_Project/V_trusteeMF_"+str(10)+".pth"), requires_grad=True)
W_trusteeMF = Variable(torch.load("drive/My Drive/SCFT_Project/W_trusteeMF_"+str(10)+".pth"), requires_grad=True)
nwi = np.load('drive/My Drive/SCFT_Project/npy_files/nwi.npy')
nvj = np.load('drive/My Drive/SCFT_Project/npy_files/nvj.npy')
mwi = np.load('drive/My Drive/SCFT_Project/npy_files/mwi.npy')
mbk = np.load('drive/My Drive/SCFT_Project/npy_files/mbk.npy')

In [ ]:
epochs = 4
alpha_lr = 0.10
optimizer = optim.SGD([B_trusteeMF, V_trusteeMF, W_trusteeMF], lr=alpha_lr)
checkpt = torch.load("drive/My Drive/SCFT_Project/optimizer_trusteeMF_5.pth")
optimizer.load_state_dict(checkpt)
print(optimizer.state_dict())
for pg in optimizer.param_groups:
    pg['lr'] = alpha_lr
print(optimizer.state_dict())

{'state': {}, 'param_groups': [{'lr': 0.4, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [139984608295456, 139984608296752, 139984608295384]}]}
{'state': {}, 'param_groups': [{'lr': 0.1, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [139984608295456, 139984608296752, 139984608295384]}]}


In [ ]:
for iteration in range(11, 11+epochs):
    start = time.time()
    loss = torch.tensor(0.0)
    optimizer.zero_grad()
    for idx in range(len(ratings_data_train)):
        user_id = int(ratings_data_train.iloc[idx]["user_id"])
        item_id = int(ratings_data_train.iloc[idx]["item_id"])
        rating = ratings_data_train.iloc[idx]["rating"]
        loss += (torch.sigmoid(torch.dot(W_trusteeMF[:, user_id-1], V_trusteeMF[:, item_id-1])) - rating).pow(2)
    for idx in range(len(trust_data)):
        user1_id = trust_data.iloc[idx]["user1_id"]
        user2_id = trust_data.iloc[idx]["user2_id"]
        trust_val = trust_data.iloc[idx]["trust_val"]
        loss += (torch.sigmoid(torch.dot(B_trusteeMF[:, user1_id-1], W_trusteeMF[:, user2_id-1])) - trust_val).pow(2)
    reg_loss = torch.tensor(0.0)
    for user_id in range(M):
        reg_loss += ((nwi[user_id]+mwi[user_id]) * (W_trusteeMF[:, user_id].pow(2).sum()))
        reg_loss += (mbk[user_id] * (B_trusteeMF[:, user_id].pow(2).sum()))
    for item_id in range(N):
        reg_loss += (nvj[item_id] * (V_trusteeMF[:, item_id].pow(2).sum()))
    loss += (lamda * reg_loss)
    loss.backward() # Loss corresponding to Eq. (5) in the paper
    optimizer.step()
    end = time.time()
    print("Iteration: ", iteration, ", loss: ", loss, ", time(seconds): ", end-start)
    torch.save(B_trusteeMF, "drive/My Drive/SCFT_Project/B_trusteeMF_"+str(iteration)+".pth")
    torch.save(V_trusteeMF, "drive/My Drive/SCFT_Project/V_trusteeMF_"+str(iteration)+".pth")
    torch.save(W_trusteeMF, "drive/My Drive/SCFT_Project/W_trusteeMF_"+str(iteration)+".pth")

Iteration:  11 , loss:  tensor(158793.0312, grad_fn=<AddBackward0>) , time(seconds):  3878.8655200004578
Iteration:  12 , loss:  tensor(96774.0391, grad_fn=<AddBackward0>) , time(seconds):  3938.159417629242
Iteration:  13 , loss:  tensor(68495.6094, grad_fn=<AddBackward0>) , time(seconds):  3906.793427467346
Iteration:  14 , loss:  tensor(56088.2305, grad_fn=<AddBackward0>) , time(seconds):  3914.5910284519196


In [ ]:
torch.save(optimizer.state_dict(), "drive/My Drive/SCFT_Project/optimizer_trusteeMF_14.pth")

In [ ]:
epochs = 4

In [ ]:
for iteration in range(15, 15+epochs):
    start = time.time()
    loss = torch.tensor(0.0)
    optimizer.zero_grad()
    for idx in range(len(ratings_data_train)):
        user_id = int(ratings_data_train.iloc[idx]["user_id"])
        item_id = int(ratings_data_train.iloc[idx]["item_id"])
        rating = ratings_data_train.iloc[idx]["rating"]
        loss += (torch.sigmoid(torch.dot(W_trusteeMF[:, user_id-1], V_trusteeMF[:, item_id-1])) - rating).pow(2)
    for idx in range(len(trust_data)):
        user1_id = trust_data.iloc[idx]["user1_id"]
        user2_id = trust_data.iloc[idx]["user2_id"]
        trust_val = trust_data.iloc[idx]["trust_val"]
        loss += (torch.sigmoid(torch.dot(B_trusteeMF[:, user1_id-1], W_trusteeMF[:, user2_id-1])) - trust_val).pow(2)
    reg_loss = torch.tensor(0.0)
    for user_id in range(M):
        reg_loss += ((nwi[user_id]+mwi[user_id]) * (W_trusteeMF[:, user_id].pow(2).sum()))
        reg_loss += (mbk[user_id] * (B_trusteeMF[:, user_id].pow(2).sum()))
    for item_id in range(N):
        reg_loss += (nvj[item_id] * (V_trusteeMF[:, item_id].pow(2).sum()))
    loss += (lamda * reg_loss)
    loss.backward() # Loss corresponding to Eq. (5) in the paper
    optimizer.step()
    end = time.time()
    print("Iteration: ", iteration, ", loss: ", loss, ", time(seconds): ", end-start)
    torch.save(B_trusteeMF, "drive/My Drive/SCFT_Project/B_trusteeMF_"+str(iteration)+".pth")
    torch.save(V_trusteeMF, "drive/My Drive/SCFT_Project/V_trusteeMF_"+str(iteration)+".pth")
    torch.save(W_trusteeMF, "drive/My Drive/SCFT_Project/W_trusteeMF_"+str(iteration)+".pth")

Iteration:  15 , loss:  tensor(49085.2188, grad_fn=<AddBackward0>) , time(seconds):  3943.584791660309
Iteration:  16 , loss:  tensor(45018.8789, grad_fn=<AddBackward0>) , time(seconds):  3860.2556190490723
Iteration:  17 , loss:  tensor(42387.2109, grad_fn=<AddBackward0>) , time(seconds):  3899.5941994190216
Iteration:  18 , loss:  tensor(41521.9141, grad_fn=<AddBackward0>) , time(seconds):  4006.6741077899933


In [ ]:
torch.save(optimizer.state_dict(), "drive/My Drive/SCFT_Project/optimizer_trusteeMF_18.pth")

In [ ]:
B_trusteeMF = Variable(torch.load("drive/My Drive/SCFT_Project/B_trusteeMF_"+str(18)+".pth"), requires_grad=True)
V_trusteeMF = Variable(torch.load("drive/My Drive/SCFT_Project/V_trusteeMF_"+str(18)+".pth"), requires_grad=True)
W_trusteeMF = Variable(torch.load("drive/My Drive/SCFT_Project/W_trusteeMF_"+str(18)+".pth"), requires_grad=True)
nwi = np.load('drive/My Drive/SCFT_Project/npy_files/nwi.npy')
nvj = np.load('drive/My Drive/SCFT_Project/npy_files/nvj.npy')
mwi = np.load('drive/My Drive/SCFT_Project/npy_files/mwi.npy')
mbk = np.load('drive/My Drive/SCFT_Project/npy_files/mbk.npy')

In [ ]:
epochs = 4
alpha_lr = 0.04
optimizer = optim.SGD([B_trusteeMF, V_trusteeMF, W_trusteeMF], lr=alpha_lr)
checkpt = torch.load("drive/My Drive/SCFT_Project/optimizer_trusteeMF_18.pth")
optimizer.load_state_dict(checkpt)
print(optimizer.state_dict())
for pg in optimizer.param_groups:
    pg['lr'] = alpha_lr
print(optimizer.state_dict())

{'state': {}, 'param_groups': [{'lr': 0.1, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [140029422903016, 140029422903160, 140029456924392]}]}
{'state': {}, 'param_groups': [{'lr': 0.04, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [140029422903016, 140029422903160, 140029456924392]}]}


In [ ]:
for iteration in range(19, 19+epochs):
    start = time.time()
    loss = torch.tensor(0.0)
    optimizer.zero_grad()
    for idx in range(len(ratings_data_train)):
        user_id = int(ratings_data_train.iloc[idx]["user_id"])
        item_id = int(ratings_data_train.iloc[idx]["item_id"])
        rating = ratings_data_train.iloc[idx]["rating"]
        loss += (torch.sigmoid(torch.dot(W_trusteeMF[:, user_id-1], V_trusteeMF[:, item_id-1])) - rating).pow(2)
    for idx in range(len(trust_data)):
        user1_id = trust_data.iloc[idx]["user1_id"]
        user2_id = trust_data.iloc[idx]["user2_id"]
        trust_val = trust_data.iloc[idx]["trust_val"]
        loss += (torch.sigmoid(torch.dot(B_trusteeMF[:, user1_id-1], W_trusteeMF[:, user2_id-1])) - trust_val).pow(2)
    reg_loss = torch.tensor(0.0)
    for user_id in range(M):
        reg_loss += ((nwi[user_id]+mwi[user_id]) * (W_trusteeMF[:, user_id].pow(2).sum()))
        reg_loss += (mbk[user_id] * (B_trusteeMF[:, user_id].pow(2).sum()))
    for item_id in range(N):
        reg_loss += (nvj[item_id] * (V_trusteeMF[:, item_id].pow(2).sum()))
    loss += (lamda * reg_loss)
    loss.backward() # Loss corresponding to Eq. (5) in the paper
    optimizer.step()
    end = time.time()
    print("Iteration: ", iteration, ", loss: ", loss, ", time(seconds): ", end-start)
    torch.save(B_trusteeMF, "drive/My Drive/SCFT_Project/B_trusteeMF_"+str(iteration)+".pth")
    torch.save(V_trusteeMF, "drive/My Drive/SCFT_Project/V_trusteeMF_"+str(iteration)+".pth")
    torch.save(W_trusteeMF, "drive/My Drive/SCFT_Project/W_trusteeMF_"+str(iteration)+".pth")

Iteration:  19 , loss:  tensor(39452.3984, grad_fn=<AddBackward0>) , time(seconds):  3930.1239247322083
Iteration:  20 , loss:  tensor(38276.7695, grad_fn=<AddBackward0>) , time(seconds):  3961.7645902633667


In [ ]:
B_trusteeMF = Variable(torch.load("drive/My Drive/SCFT_Project/B_trusteeMF_"+str(20)+".pth"), requires_grad=True)
V_trusteeMF = Variable(torch.load("drive/My Drive/SCFT_Project/V_trusteeMF_"+str(20)+".pth"), requires_grad=True)
W_trusteeMF = Variable(torch.load("drive/My Drive/SCFT_Project/W_trusteeMF_"+str(20)+".pth"), requires_grad=True)
nwi = np.load('drive/My Drive/SCFT_Project/npy_files/nwi.npy')
nvj = np.load('drive/My Drive/SCFT_Project/npy_files/nvj.npy')
mwi = np.load('drive/My Drive/SCFT_Project/npy_files/mwi.npy')
mbk = np.load('drive/My Drive/SCFT_Project/npy_files/mbk.npy')

In [ ]:
epochs = 4
alpha_lr = 0.08
optimizer = optim.SGD([B_trusteeMF, V_trusteeMF, W_trusteeMF], lr=alpha_lr)
checkpt = torch.load("drive/My Drive/SCFT_Project/optimizer_trusteeMF_18.pth")
optimizer.load_state_dict(checkpt)
print(optimizer.state_dict())
for pg in optimizer.param_groups:
    pg['lr'] = alpha_lr
print(optimizer.state_dict())

{'state': {}, 'param_groups': [{'lr': 0.1, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [140597119802032, 140597119824808, 140597119825168]}]}
{'state': {}, 'param_groups': [{'lr': 0.08, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [140597119802032, 140597119824808, 140597119825168]}]}


In [ ]:
for iteration in range(21, 21+epochs):
    start = time.time()
    loss = torch.tensor(0.0)
    optimizer.zero_grad()
    for idx in range(len(ratings_data_train)):
        user_id = int(ratings_data_train.iloc[idx]["user_id"])
        item_id = int(ratings_data_train.iloc[idx]["item_id"])
        rating = ratings_data_train.iloc[idx]["rating"]
        loss += (torch.sigmoid(torch.dot(W_trusteeMF[:, user_id-1], V_trusteeMF[:, item_id-1])) - rating).pow(2)
    for idx in range(len(trust_data)):
        user1_id = trust_data.iloc[idx]["user1_id"]
        user2_id = trust_data.iloc[idx]["user2_id"]
        trust_val = trust_data.iloc[idx]["trust_val"]
        loss += (torch.sigmoid(torch.dot(B_trusteeMF[:, user1_id-1], W_trusteeMF[:, user2_id-1])) - trust_val).pow(2)
    reg_loss = torch.tensor(0.0)
    for user_id in range(M):
        reg_loss += ((nwi[user_id]+mwi[user_id]) * (W_trusteeMF[:, user_id].pow(2).sum()))
        reg_loss += (mbk[user_id] * (B_trusteeMF[:, user_id].pow(2).sum()))
    for item_id in range(N):
        reg_loss += (nvj[item_id] * (V_trusteeMF[:, item_id].pow(2).sum()))
    loss += (lamda * reg_loss)
    loss.backward() # Loss corresponding to Eq. (5) in the paper
    optimizer.step()
    end = time.time()
    print("Iteration: ", iteration, ", loss: ", loss, ", time(seconds): ", end-start)
    torch.save(B_trusteeMF, "drive/My Drive/SCFT_Project/B_trusteeMF_"+str(iteration)+".pth")
    torch.save(V_trusteeMF, "drive/My Drive/SCFT_Project/V_trusteeMF_"+str(iteration)+".pth")
    torch.save(W_trusteeMF, "drive/My Drive/SCFT_Project/W_trusteeMF_"+str(iteration)+".pth")

Iteration:  21 , loss:  tensor(37322.4219, grad_fn=<AddBackward0>) , time(seconds):  4213.098088264465
Iteration:  22 , loss:  tensor(35904.5938, grad_fn=<AddBackward0>) , time(seconds):  4264.500135183334
Iteration:  23 , loss:  tensor(35893.6641, grad_fn=<AddBackward0>) , time(seconds):  4189.6291427612305
Iteration:  24 , loss:  tensor(34901.6406, grad_fn=<AddBackward0>) , time(seconds):  4171.842748403549


In [ ]:
torch.save(optimizer.state_dict(), "drive/My Drive/SCFT_Project/optimizer_trusteeMF_24.pth")

In [ ]:
B_trusteeMF = Variable(torch.load("drive/My Drive/SCFT_Project/B_trusteeMF_"+str(24)+".pth"), requires_grad=True)
V_trusteeMF = Variable(torch.load("drive/My Drive/SCFT_Project/V_trusteeMF_"+str(24)+".pth"), requires_grad=True)
W_trusteeMF = Variable(torch.load("drive/My Drive/SCFT_Project/W_trusteeMF_"+str(24)+".pth"), requires_grad=True)
nwi = np.load('drive/My Drive/SCFT_Project/npy_files/nwi.npy')
nvj = np.load('drive/My Drive/SCFT_Project/npy_files/nvj.npy')
mwi = np.load('drive/My Drive/SCFT_Project/npy_files/mwi.npy')
mbk = np.load('drive/My Drive/SCFT_Project/npy_files/mbk.npy')

In [ ]:
epochs = 4
alpha_lr = 0.08
optimizer = optim.SGD([B_trusteeMF, V_trusteeMF, W_trusteeMF], lr=alpha_lr)
checkpt = torch.load("drive/My Drive/SCFT_Project/optimizer_trusteeMF_24.pth")
optimizer.load_state_dict(checkpt)
print(optimizer.state_dict())
for pg in optimizer.param_groups:
    pg['lr'] = alpha_lr
print(optimizer.state_dict())

{'state': {}, 'param_groups': [{'lr': 0.08, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [139986776931928, 139986743175712, 139988371950688]}]}
{'state': {}, 'param_groups': [{'lr': 0.08, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [139986776931928, 139986743175712, 139988371950688]}]}


In [ ]:
for iteration in range(25, 25+epochs):
    start = time.time()
    loss = torch.tensor(0.0)
    optimizer.zero_grad()
    for idx in range(len(ratings_data_train)):
        user_id = int(ratings_data_train.iloc[idx]["user_id"])
        item_id = int(ratings_data_train.iloc[idx]["item_id"])
        rating = ratings_data_train.iloc[idx]["rating"]
        loss += (torch.sigmoid(torch.dot(W_trusteeMF[:, user_id-1], V_trusteeMF[:, item_id-1])) - rating).pow(2)
    for idx in range(len(trust_data)):
        user1_id = trust_data.iloc[idx]["user1_id"]
        user2_id = trust_data.iloc[idx]["user2_id"]
        trust_val = trust_data.iloc[idx]["trust_val"]
        loss += (torch.sigmoid(torch.dot(B_trusteeMF[:, user1_id-1], W_trusteeMF[:, user2_id-1])) - trust_val).pow(2)
    reg_loss = torch.tensor(0.0)
    for user_id in range(M):
        reg_loss += ((nwi[user_id]+mwi[user_id]) * (W_trusteeMF[:, user_id].pow(2).sum()))
        reg_loss += (mbk[user_id] * (B_trusteeMF[:, user_id].pow(2).sum()))
    for item_id in range(N):
        reg_loss += (nvj[item_id] * (V_trusteeMF[:, item_id].pow(2).sum()))
    loss += (lamda * reg_loss)
    loss.backward() # Loss corresponding to Eq. (5) in the paper
    optimizer.step()
    end = time.time()
    print("Iteration: ", iteration, ", loss: ", loss, ", time(seconds): ", end-start)
    torch.save(B_trusteeMF, "drive/My Drive/SCFT_Project/B_trusteeMF_"+str(iteration)+".pth")
    torch.save(V_trusteeMF, "drive/My Drive/SCFT_Project/V_trusteeMF_"+str(iteration)+".pth")
    torch.save(W_trusteeMF, "drive/My Drive/SCFT_Project/W_trusteeMF_"+str(iteration)+".pth")

Iteration:  25 , loss:  tensor(34010.9922, grad_fn=<AddBackward0>) , time(seconds):  4018.45547246933
Iteration:  26 , loss:  tensor(33434.8750, grad_fn=<AddBackward0>) , time(seconds):  4229.299179315567
Iteration:  27 , loss:  tensor(32953.0547, grad_fn=<AddBackward0>) , time(seconds):  4185.780665874481
Iteration:  28 , loss:  tensor(32059.2109, grad_fn=<AddBackward0>) , time(seconds):  4058.7082839012146


In [ ]:
torch.save(optimizer.state_dict(), "drive/My Drive/SCFT_Project/optimizer_trusteeMF_28.pth")

In [ ]:
epochs = 4
alpha_lr = 0.1
print(optimizer.state_dict())
for pg in optimizer.param_groups:
    pg['lr'] = alpha_lr
print(optimizer.state_dict())

{'state': {}, 'param_groups': [{'lr': 0.08, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [139986776931928, 139986743175712, 139988371950688]}]}
{'state': {}, 'param_groups': [{'lr': 0.1, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [139986776931928, 139986743175712, 139988371950688]}]}


In [ ]:
for iteration in range(29, 29+epochs):
    start = time.time()
    loss = torch.tensor(0.0)
    optimizer.zero_grad()
    for idx in range(len(ratings_data_train)):
        user_id = int(ratings_data_train.iloc[idx]["user_id"])
        item_id = int(ratings_data_train.iloc[idx]["item_id"])
        rating = ratings_data_train.iloc[idx]["rating"]
        loss += (torch.sigmoid(torch.dot(W_trusteeMF[:, user_id-1], V_trusteeMF[:, item_id-1])) - rating).pow(2)
    for idx in range(len(trust_data)):
        user1_id = trust_data.iloc[idx]["user1_id"]
        user2_id = trust_data.iloc[idx]["user2_id"]
        trust_val = trust_data.iloc[idx]["trust_val"]
        loss += (torch.sigmoid(torch.dot(B_trusteeMF[:, user1_id-1], W_trusteeMF[:, user2_id-1])) - trust_val).pow(2)
    reg_loss = torch.tensor(0.0)
    for user_id in range(M):
        reg_loss += ((nwi[user_id]+mwi[user_id]) * (W_trusteeMF[:, user_id].pow(2).sum()))
        reg_loss += (mbk[user_id] * (B_trusteeMF[:, user_id].pow(2).sum()))
    for item_id in range(N):
        reg_loss += (nvj[item_id] * (V_trusteeMF[:, item_id].pow(2).sum()))
    loss += (lamda * reg_loss)
    loss.backward() # Loss corresponding to Eq. (5) in the paper
    optimizer.step()
    end = time.time()
    print("Iteration: ", iteration, ", loss: ", loss, ", time(seconds): ", end-start)
    torch.save(B_trusteeMF, "drive/My Drive/SCFT_Project/B_trusteeMF_"+str(iteration)+".pth")
    torch.save(V_trusteeMF, "drive/My Drive/SCFT_Project/V_trusteeMF_"+str(iteration)+".pth")
    torch.save(W_trusteeMF, "drive/My Drive/SCFT_Project/W_trusteeMF_"+str(iteration)+".pth")

Iteration:  29 , loss:  tensor(31730.7695, grad_fn=<AddBackward0>) , time(seconds):  3991.024946451187
Iteration:  30 , loss:  tensor(31726.9453, grad_fn=<AddBackward0>) , time(seconds):  4250.053227901459
Iteration:  31 , loss:  tensor(31635.4590, grad_fn=<AddBackward0>) , time(seconds):  4189.5547025203705
Iteration:  32 , loss:  tensor(31092.6797, grad_fn=<AddBackward0>) , time(seconds):  4157.575653791428


In [ ]:
torch.save(optimizer.state_dict(), "drive/My Drive/SCFT_Project/optimizer_trusteeMF_32.pth")

In [ ]:
B_trusteeMF = Variable(torch.load("drive/My Drive/SCFT_Project/B_trusteeMF_"+str(32)+".pth"), requires_grad=True)
V_trusteeMF = Variable(torch.load("drive/My Drive/SCFT_Project/V_trusteeMF_"+str(32)+".pth"), requires_grad=True)
W_trusteeMF = Variable(torch.load("drive/My Drive/SCFT_Project/W_trusteeMF_"+str(32)+".pth"), requires_grad=True)
nwi = np.load('drive/My Drive/SCFT_Project/npy_files/nwi.npy')
nvj = np.load('drive/My Drive/SCFT_Project/npy_files/nvj.npy')
mwi = np.load('drive/My Drive/SCFT_Project/npy_files/mwi.npy')
mbk = np.load('drive/My Drive/SCFT_Project/npy_files/mbk.npy')

In [ ]:
epochs = 2
alpha_lr = 0.2
optimizer = optim.SGD([B_trusteeMF, V_trusteeMF, W_trusteeMF], lr=alpha_lr)
checkpt = torch.load("drive/My Drive/SCFT_Project/optimizer_trusteeMF_32.pth")
optimizer.load_state_dict(checkpt)
print(optimizer.state_dict())
for pg in optimizer.param_groups:
    pg['lr'] = alpha_lr
print(optimizer.state_dict())

{'state': {}, 'param_groups': [{'lr': 0.1, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [140608972485472, 140610584456288, 140608972486048]}]}
{'state': {}, 'param_groups': [{'lr': 0.2, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [140608972485472, 140610584456288, 140608972486048]}]}


In [ ]:
for iteration in range(33, 34):
    start = time.time()
    loss = torch.tensor(0.0)
    optimizer.zero_grad()
    for idx in range(len(ratings_data_train)):
        user_id = int(ratings_data_train.iloc[idx]["user_id"])
        item_id = int(ratings_data_train.iloc[idx]["item_id"])
        rating = ratings_data_train.iloc[idx]["rating"]
        loss += (torch.sigmoid(torch.dot(W_trusteeMF[:, user_id-1], V_trusteeMF[:, item_id-1])) - rating).pow(2)
    for idx in range(len(trust_data)):
        user1_id = trust_data.iloc[idx]["user1_id"]
        user2_id = trust_data.iloc[idx]["user2_id"]
        trust_val = trust_data.iloc[idx]["trust_val"]
        loss += (torch.sigmoid(torch.dot(B_trusteeMF[:, user1_id-1], W_trusteeMF[:, user2_id-1])) - trust_val).pow(2)
    reg_loss = torch.tensor(0.0)
    for user_id in range(M):
        reg_loss += ((nwi[user_id]+mwi[user_id]) * (W_trusteeMF[:, user_id].pow(2).sum()))
        reg_loss += (mbk[user_id] * (B_trusteeMF[:, user_id].pow(2).sum()))
    for item_id in range(N):
        reg_loss += (nvj[item_id] * (V_trusteeMF[:, item_id].pow(2).sum()))
    loss += (lamda * reg_loss)
    loss.backward() # Loss corresponding to Eq. (5) in the paper
    optimizer.step()
    end = time.time()
    print("Iteration: ", iteration, ", loss: ", loss, ", time(seconds): ", end-start)
    torch.save(B_trusteeMF, "drive/My Drive/SCFT_Project/B_trusteeMF_"+str(iteration)+".pth")
    torch.save(V_trusteeMF, "drive/My Drive/SCFT_Project/V_trusteeMF_"+str(iteration)+".pth")
    torch.save(W_trusteeMF, "drive/My Drive/SCFT_Project/W_trusteeMF_"+str(iteration)+".pth")

Iteration:  33 , loss:  tensor(200326.6094, grad_fn=<AddBackward0>) , time(seconds):  4621.4159190654755


In [ ]:
torch.save(optimizer.state_dict(), "drive/My Drive/SCFT_Project/optimizer_trusteeMF_34.pth")

### Predict Rating:

In [ ]:
def get_model_parameters(iteration_r, iteration_e):
    B_trusterMF = Variable(torch.load("drive/My Drive/SCFT_Project/B_trusterMF_"+str(iteration_r)+".pth"), requires_grad=False)
    V_trusterMF = Variable(torch.load("drive/My Drive/SCFT_Project/V_trusterMF_"+str(iteration_r)+".pth"), requires_grad=False)
    W_trusteeMF = Variable(torch.load("drive/My Drive/SCFT_Project/W_trusteeMF_"+str(iteration_e)+".pth"), requires_grad=False)
    V_trusteeMF = Variable(torch.load("drive/My Drive/SCFT_Project/V_trusteeMF_"+str(iteration_e)+".pth"), requires_grad=False)
    model_dict = {'B_trusterMF': B_trusterMF, 'V_trusterMF': V_trusterMF, 'W_trusteeMF': W_trusteeMF, 'V_trusteeMF': V_trusteeMF}
    return model_dict

In [ ]:
def predict_rating(model_dict, user_id, item_id):
    global R_max 
    B_trusterMF = model_dict['B_trusterMF']
    V_trusterMF = model_dict['V_trusterMF']
    W_trusteeMF = model_dict['W_trusteeMF']
    V_trusteeMF = model_dict['V_trusteeMF']
    prediction = R_max * (torch.sigmoid((torch.dot(B_trusterMF[:, user_id-1], V_trusterMF[:, item_id-1]) + torch.dot(W_trusteeMF[:, user_id-1], V_trusteeMF[:, item_id-1])) / 2))
    return prediction
    # return prediction.data

### Validation on All Users (Paper Section 4.3.1):

In [ ]:
# Mean Absolute Error
def compute_MAE(model_dict, mode="train"):
    global ratings_data_train, ratings_data_test
    if mode == "train":
        data = ratings_data_train
    else:
        data = ratings_data_test
    AE = torch.tensor(0.0)
    for idx in range(len(data)):
        user_id = int(data.iloc[idx]["user_id"])
        item_id = int(data.iloc[idx]["item_id"])
        rating = (data.iloc[idx]["rating"]) * R_max
        AE += torch.abs(predict_rating(model_dict, user_id, item_id) - rating)
    MAE = AE / len(data)
    return MAE

In [ ]:
# Root Mean Square Error
def compute_RMSE(model_dict, mode="train"):
    global ratings_data_train, ratings_data_test
    if mode == "train":
        data = ratings_data_train
    else:
        data = ratings_data_test
    SE = torch.tensor(0.0)
    for idx in range(len(data)):
        user_id = int(data.iloc[idx]["user_id"])
        item_id = int(data.iloc[idx]["item_id"])
        rating = (data.iloc[idx]["rating"]) * R_max
        SE += ((predict_rating(model_dict, user_id, item_id) - rating).pow(2))
    MSE = SE / len(data)
    RMSE = torch.sqrt(MSE)
    return RMSE

In [ ]:
model_dict = get_model_parameters(15, 33)

In [ ]:
print(compute_MAE(model_dict, mode="train"))

tensor(1.7545)


In [ ]:
print(compute_MAE(model_dict, mode="test"))

tensor(1.7665)


In [ ]:
print(compute_RMSE(model_dict, mode="train"))

tensor(1.9099)


In [ ]:
print(compute_RMSE(model_dict, mode="test"))

tensor(1.9220)


### Validation on Cold Start Users (Paper Section 4.3.2):

In [ ]:
# Mean Absolute Error For Cold Start Users
def compute_MAE_cold_start(model_dict, mode="train", threshold=5):
    global ratings_data_train, ratings_data_test
    nbi = np.load('drive/My Drive/SCFT_Project/npy_files/nbi.npy')
    if mode == "train":
        data = ratings_data_train
    else:
        data = ratings_data_test
    AE = torch.tensor(0.0)
    cnt = 0
    for idx in range(len(data)):
        user_id = int(data.iloc[idx]["user_id"])
        if nbi[user_id-1] > threshold: # NOT A COLD START USER
            continue
        item_id = int(data.iloc[idx]["item_id"])
        rating = (data.iloc[idx]["rating"]) * R_max
        cnt += 1
        AE += torch.abs(predict_rating(model_dict, user_id, item_id) - rating)
    MAE = AE / cnt
    return MAE

In [ ]:
# Root Mean Square Error For Cold Start Users
def compute_RMSE_cold_start(model_dict, mode="train", threshold=5):
    global ratings_data_train, ratings_data_test
    nbi = np.load('drive/My Drive/SCFT_Project/npy_files/nbi.npy')
    if mode == "train":
        data = ratings_data_train
    else:
        data = ratings_data_test
    SE = torch.tensor(0.0)
    cnt = 0
    for idx in range(len(data)):
        user_id = int(data.iloc[idx]["user_id"])
        if nbi[user_id-1] > threshold: # NOT A COLD START USER
            continue
        item_id = int(data.iloc[idx]["item_id"])
        rating = (data.iloc[idx]["rating"]) * R_max
        cnt += 1
        SE += ((predict_rating(model_dict, user_id, item_id) - rating).pow(2))
    MSE = SE / cnt
    RMSE = torch.sqrt(MSE)
    return RMSE

In [ ]:
model_dict = get_model_parameters(15, 33)

In [ ]:
print(compute_MAE_cold_start(model_dict, mode="train"))

tensor(1.8496)


In [ ]:
print(compute_MAE_cold_start(model_dict, mode="test"))

tensor(1.8444)


In [ ]:
print(compute_RMSE_cold_start(model_dict, mode="train"))

tensor(1.9878)


In [ ]:
print(compute_RMSE_cold_start(model_dict, mode="test"))

tensor(1.9855)


### References:

##### Epinions Dataset:
http://www.trustlet.org/downloaded_epinions.html  